# Receiving weather API data and inserting into Database

## This projects use following Libraries:
- requests==2.31.0
- pandas==2.1.4
- geopandas==0.14.1
- DateTime==5.4
- matplotlib==3.8.2
- folium==0.15.1
- mapclassify==2.6.1


In [1]:
import os
import requests
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import datetime
import matplotlib.pyplot as plt
import numpy as np
import json

## API KEY From https://openweathermap.org/

In [2]:
API_KEY = "a3a03b31b2fcfa0091ebdd5bcd9eaabe"

## Make variable to input city name

In [3]:
# city = input("Enter a city name: ")
city = "Jakarta"

## Building code for getting API to JSON

In [4]:
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
request_url = f"{BASE_URL}?appid={API_KEY}&q={city}"
response = requests.get(request_url)

if response.status_code == 200:
    data = response.json()
    print("Data Accepted.")
else:
    print("An error occured.")



Data Accepted.


## Cleansing JSON Data

In [5]:
# analyze the response API data
print(data)

{'coord': {'lon': 106.8451, 'lat': -6.2146}, 'weather': [{'id': 211, 'main': 'Thunderstorm', 'description': 'thunderstorm', 'icon': '11d'}], 'base': 'stations', 'main': {'temp': 305.05, 'feels_like': 312.05, 'temp_min': 304.16, 'temp_max': 305.2, 'pressure': 1009, 'humidity': 68}, 'visibility': 5000, 'wind': {'speed': 3.09, 'deg': 360}, 'clouds': {'all': 75}, 'dt': 1703665227, 'sys': {'type': 1, 'id': 9383, 'country': 'ID', 'sunrise': 1703630328, 'sunset': 1703675259}, 'timezone': 25200, 'id': 1642911, 'name': 'Jakarta', 'cod': 200}


In [6]:
# Get Coordinate
Lat = data['coord']['lat']
Long = data['coord']['lon']

# Convert Unix time to Datetime
Date = str(datetime.datetime.fromtimestamp(data['dt']).strftime('%Y-%m-%d %H:%M:%S'))

# Get weather data
City_Name = data['name']
Temp_Avg = round(((data['main']['temp_max']+data['main']['temp_min'])/2)- 273.15,2)
Pressure = data['main']['pressure']
Humidity = data['main']['humidity']
Wind_spd = data['wind']['speed']
Wind_deg = data['wind']['deg']
Cloud_Percent = data['clouds']['all']

### Validating the data

In [7]:
Data_Clean = pd.DataFrame({
    'Date': Date,
    'City': City_Name,
    'Temp_Avg': str(Temp_Avg) + " Celcius",
    'Pressure': Pressure,
    'Humidity': str(Humidity) + " %",
    'Wind_spd': Wind_spd,
    'Wind_deg': Wind_deg,
    'Cloud': str(Cloud_Percent) + " %",
    'Long': Long,
    'Lat': Lat,}
    , index=[0])

print("After cleansing data from API.")
print(Data_Clean)

After cleansing data from API.
                  Date     City       Temp_Avg  Pressure Humidity  Wind_spd  \
0  2023-12-27 15:20:27  Jakarta  31.53 Celcius      1009     68 %      3.09   

   Wind_deg Cloud      Long     Lat  
0       360  75 %  106.8451 -6.2146  


### Visualize data from API

In [8]:
# Location
crs = {'init':'epsg:4326'}
geo_data = Data_Clean
Data_Clean['geometry'] = gpd.points_from_xy(Data_Clean['Long'], Data_Clean['Lat'])
geo_data = gpd.GeoDataFrame(Data_Clean, crs=crs, geometry=Data_Clean['geometry'])
geo_data.explore()
# Wind Speed and direction

# Temperature, Pressure, Humidity, and Cloud

c:\Project Coding\Python\weather API\.venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


## inserting to Databases

In [9]:
# result = Data_Clean.to_json(r'datastorage\weatherdata.json', orient='records', lines=True, default=str)

# Convert the DataFrame to a dictionary and then to JSON
if 'geometry' in Data_Clean.columns:
    Data_Clean = pd.DataFrame(Data_Clean.drop(columns='geometry'))
JSON_Data = Data_Clean.to_dict(orient='records')

# Export JSON data to a file
try:
    file_path = 'datastorage/weatherdata.json'
    # Checking weather data storage
    if os.path.exists(file_path):
        print("File exists. Appending data.")
        Data_Existing = pd.read_json(file_path)
        JSON_Existing = Data_Existing.to_dict()
        JSON_Concat = JSON_Existing.update(JSON_Data)

        # Data_Existing = pd.read_json(file_path)
        # print(Data_Existing)
        # Data_Concat = pd.concat([Data_Existing, Data_Clean])
        # print(Data_Concat)
        # JSON_Concat = Data_Concat.to_dict(orient='records')
        # print(JSON_Concat)
        with open(file_path, 'w') as JSON_File:
            json.dump(JSON_Concat, JSON_File)
        print("DataFrame exported to JSON successfully.")

    # if data not available, make a new one
    else:
        with open(file_path, 'w') as JSON_File:
            json.dump(JSON_Data, JSON_File)
        print("DataFrame exported to JSON successfully.")

# Printing error that occured if existed
except Exception as e:
    print(f"Error: {e}")

DataFrame exported to JSON successfully.


## Visualizing time series data from databases

### Load JSON format data and covert into DataFrame

In [ ]:
Dataset = pd.read_json(file_path)
print(Dataset)


### Building geometry to visualize lat,long dataset

In [ ]:
# Define CRS
CRS = {'init':'epsg:4326'}

Dataset['geometry'] = gpd.points_from_xy(Dataset['Long'], Dataset['Lat'])
Dataset = gpd.GeoDataFrame(Dataset, crs=crs, geometry=Dataset['geometry'])
Dataset.explore()

### Filtering dataset for specific city/location

In [ ]:
Filter_City = input('Input a City: ')
Dataset_Filtered = Dataset[Dataset['City'] == 'Jakarta']
# Dataset_Filtered = Dataset[Dataset['City'] == 'Jakarta']
print(Dataset_Filtered)

if Dataset_Filtered == False:
    print("Data not available.")
else:
    pass

### Plotting by date with Temperature, Humidity, Pressure, and Cloud Visibility